In [1]:
using LinearAlgebra # Nioce.
# open file 
mni = open("input.txt","r");
raw = readlines(mni)
scans = []
cur_scan = Matrix{Int}(undef, 0, 3)
for line in raw
    if line == ""
        push!(scans, cur_scan)
        cur_scan = Matrix{Int}(undef, 0, 3)
        continue
    elseif line[1:3] == "---"
        continue
    end
    cur_scan = [cur_scan; reshape([parse(Int, val) for val in split(line,",")],1,3)]
end
  
push!(scans, cur_scan);

In [2]:
function Rx(θ)
    rot = zeros(Int,3,3)
    rot[1,1] = 1
    rot[2,2] = cosd(θ)
    rot[2,3] = sind(θ)
    rot[3,2] = -sind(θ)
    rot[3,3] = cosd(θ)
    return rot
end
function Ry(θ)
    rot = zeros(Int,3,3)
    rot[1,1] = cosd(θ)
    rot[1,3] = -sind(θ)
    rot[2,2] = 1
    rot[3,1] = sind(θ)
    rot[3,3] = cosd(θ)
    return rot
end
function Rz(θ)
    rot = zeros(Int,3,3)
    rot[1,1] = cosd(θ)
    rot[1,2] = sind(θ)
    rot[2,1] = -sind(θ)
    rot[2,2] = cosd(θ)
    rot[3,3] = 1
    return rot
end

Rz (generic function with 1 method)

In [3]:
const Rxc = Rx(90)
const Ryc = Ry(90)
const Rzc = Rz(90)
const neg_x = [-1 0 0; 0 1 0; 0 0 1]
const neg_y = [1 0 0; 0 -1 0; 0 0 1]
const neg_z = [1 0 0; 0 1 0; 0 0 -1];

In [4]:
function translate(scan, t)
    T = t .* ones(Int, size(scan, 1), size(scan, 2))
    t_scan = scan .+ T
    return t_scan
end

function overlap(scan, full_map)
    overlap = Matrix{Int}(undef, 0, 3)
    for s in 1:size(scan, 1)
        scan_pt = transpose(scan[s,1:3])
        for m in 1:size(full_map, 1)
            map_pt = transpose(full_map[m,1:3])
            
            if scan_pt == map_pt
                overlap = [overlap; scan_pt]
            end
        end
    end
    
    return overlap
end
function get_translations(scan, full_map)
    translations = []
    for s in 1:size(scan, 1)
        scan_pt = transpose(scan[s,1:3])
        for m in 1:size(full_map, 1)
            map_pt = transpose(full_map[m,1:3])
            T = map_pt - scan_pt
            push!(translations, T)
        end
    end
    return translations
end
function get_rotations()
    rotations = []
    push!(rotations, I)
    push!(rotations, Rzc)
    push!(rotations, Rzc*Rzc)
    push!(rotations, Rzc*Rzc*Rzc)
    push!(rotations, Ryc)
    push!(rotations, Ryc*Ryc*Ryc)
    
    push!(rotations, Rxc)
    push!(rotations, Rxc*Rzc)
    push!(rotations, Rxc*Rzc*Rzc)
    push!(rotations, Rxc*Rzc*Rzc*Rzc)
    push!(rotations, Rxc*Ryc)
    push!(rotations, Rxc*Ryc*Ryc*Ryc)
    
    push!(rotations, Rxc*Rxc)
    push!(rotations, Rxc*Rxc*Rzc)
    push!(rotations, Rxc*Rxc*Rzc*Rzc)
    push!(rotations, Rxc*Rxc*Rzc*Rzc*Rzc)
    push!(rotations, Rxc*Rxc*Ryc)
    push!(rotations, Rxc*Rxc*Ryc*Ryc*Ryc)
    
    push!(rotations, Rxc*Rxc*Rxc)
    push!(rotations, Rxc*Rxc*Rxc*Rzc)
    push!(rotations, Rxc*Rxc*Rxc*Rzc*Rzc)
    push!(rotations, Rxc*Rxc*Rxc*Rzc*Rzc*Rzc)
    push!(rotations, Rxc*Rxc*Rxc*Ryc)
    push!(rotations, Rxc*Rxc*Rxc*Ryc*Ryc*Ryc)
    
    return rotations
end
function merge_scan(scan, full_map)
    new_map = copy(full_map)
    
    for s in 1:size(scan, 1)
        scan_pt = transpose(scan[s,1:3])
        exists = false
        for m in 1:size(full_map, 1)
            map_pt = transpose(full_map[m,1:3])
            
            if scan_pt == map_pt
                exists = true
                break
            end
        end
        
        if !exists
           new_map = [new_map; scan_pt]
        end
    end
    return new_map
end
function register_scan(scan, full_map)
    full_map = copy(full_map)
    rotation = I
    translation = I
    for R in get_rotations()
        rotated = scan*R
        translations = get_translations(rotated, full_map)
        for T in translations
            #@show T
            rot_trans_scan = translate(rotated, T)
            common_beacons =  overlap(rot_trans_scan, full_map)
            if length(common_beacons) >= 12*3
                full_map = merge_scan(rot_trans_scan, full_map)
                rotation = R
                translation = T
                return full_map, rotation, translation
            end
        end
     end
    return full_map, rotation, translation
end

register_scan (generic function with 1 method)

In [5]:
ss = copy(scans)
full_map = popfirst!(scans)
registered_scans = []

while length(ss) > 1
    scan = popfirst!(ss)
    new_map, R, T = register_scan(scan, full_map);
    if new_map == full_map
        push!(ss, scan)
    else
        full_map = new_map
    end
end

In [6]:
@show length(full_map)/3
for i in 1:size(full_map, 1)
    @show full_map[i:i+2]
end

length(full_map) / 3 = 320.0
full_map[i:i + 2] = [-750, 651, 545]
full_map[i:i + 2] = [651, 545, -869]
full_map[i:i + 2] = [545, -869, -709]
full_map[i:i + 2] = [-869, -709, 391]
full_map[i:i + 2] = [-709, 391, -748]
full_map[i:i + 2] = [391, -748, 585]
full_map[i:i + 2] = [-748, 585, 618]
full_map[i:i + 2] = [585, 618, -359]
full_map[i:i + 2] = [618, -359, -738]
full_map[i:i + 2] = [-359, -738, -844]
full_map[i:i + 2] = [-738, -844, 466]
full_map[i:i + 2] = [-844, 466, -781]
full_map[i:i + 2] = [466, -781, -548]
full_map[i:i + 2] = [-781, -548, -118]
full_map[i:i + 2] = [-548, -118, 333]
full_map[i:i + 2] = [-118, 333, -703]
full_map[i:i + 2] = [333, -703, -866]
full_map[i:i + 2] = [-703, -866, 598]
full_map[i:i + 2] = [-866, 598, 491]
full_map[i:i + 2] = [598, 491, 661]
full_map[i:i + 2] = [491, 661, 320]
full_map[i:i + 2] = [661, 320, 35]
full_map[i:i + 2] = [320, 35, 586]
full_map[i:i + 2] = [35, 586, -505]
full_map[i:i + 2] = [586, -505, 846]
full_map[i:i + 2] = [-505, 846, -124]


full_map[i:i + 2] = [723, 800, -582]
full_map[i:i + 2] = [800, -582, 866]
full_map[i:i + 2] = [-582, 866, 857]
full_map[i:i + 2] = [866, 857, -88]
full_map[i:i + 2] = [857, -88, -499]
full_map[i:i + 2] = [-88, -499, -739]
full_map[i:i + 2] = [-499, -739, -648]
full_map[i:i + 2] = [-739, -648, 701]
full_map[i:i + 2] = [-648, 701, -659]
full_map[i:i + 2] = [701, -659, 790]
full_map[i:i + 2] = [-659, 790, -563]
full_map[i:i + 2] = [790, -563, 320]
full_map[i:i + 2] = [-563, 320, -449]
full_map[i:i + 2] = [320, -449, -477]
full_map[i:i + 2] = [-449, -477, 344]
full_map[i:i + 2] = [-477, 344, 536]
full_map[i:i + 2] = [344, 536, 653]
full_map[i:i + 2] = [536, 653, -470]
full_map[i:i + 2] = [653, -470, 296]
full_map[i:i + 2] = [-470, 296, 65]
full_map[i:i + 2] = [296, 65, -534]
full_map[i:i + 2] = [65, -534, -76]
full_map[i:i + 2] = [-534, -76, -578]
full_map[i:i + 2] = [-76, -578, 758]
full_map[i:i + 2] = [-578, 758, -1585]
full_map[i:i + 2] = [758, -1585, -1586]
full_map[i:i + 2] = [-1585, 

In [7]:
length(full_map)/3

320.0